# Introduction to DEEPLAY for Developers

This notebook presents a minimal example of how to implement a neural network and and an application with deeplay.
Specifically, it implements the classes for a multilayer perceptron and a classifier.
Then, it combines them to demonstrate how tehy can be used for a simple classification task.
Finally, it upgrades these classes adding functionalities that are required to improved the user experience when using an IDE.

## Minimal Multilayer Perceptron

Here, we implement the minimal class `SimpleMLP` that represents a multilayer perceptron with a certain umber of inputs (`ìn_features`, which is an integer), a series of hidden layers with a certain number of neurons (`hidden_features`, a vector with the number of neurons for each layer), and a certain numebr of outputs (`out_features`, which is an integer).

The constructor initializes the MLP by creating a sequence of linear and ReLU activation layers.

The `forward` method defines the data flow through the network, sequentially passing the input through each linear-activation block and returning the final output.

In [ ]:
import deeplay as dl
import torch.nn as nn

class SimpleMLP(dl.DeeplayModule):

    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        
        self.blocks = dl.LayerList()
        for inputs_layer, outputs_layer in zip([in_features, *hidden_features], 
                                               [*hidden_features, out_features]):
            self.blocks.append(
                dl.LayerActivationBlock(
                    dl.Layer(nn.Linear, inputs_layer, outputs_layer),
                    dl.Layer(nn.ReLU)
                )
            )

    def forward(self, x):
        for block in self.blocks:
            x = block(x)

        return x

We can now create an instance of `SimpleMLP` in various ways, for example:

```python
mlp = SimpleMLP(2, [32, 32], 2)
```

or more explicitly:

```python
mlp = SimpleMLP(
    in_features=2, 
    hidden_feature=[32, 32], 
    out_features=2
)
```

In [ ]:
mlp = SimpleMLP(2, [32, 32], 2)

print(mlp)

## Minimal Classifier

In [ ]:
# Minimal classifier
import torchmetrics as tm

class Classifier(dl.Application):

    def __init__(self, model, **kwargs):
        self.model = model
        super().__init__(**kwargs)
        
        
    def forward(self, x):
        return self.model(x)
    
classifier = Classifier(
    mlp, 
    loss=nn.CrossEntropyLoss(), 
    optimizer=dl.Adam(lr=1e-3), 
    metrics=[tm.Accuracy("multiclass", num_classes=2)]
)

classifier.model.blocks[-1].activation.configure(nn.Identity)
classifier.build()

## Example

In [ ]:
# example:

import torch
n_samples = 10000
train_data = torch.rand(n_samples, 2)
train_labels = (train_data.sum(dim=1) > 1).long()

dataset = torch.utils.data.TensorDataset(train_data, train_labels)
train, val = torch.utils.data.random_split(dataset, [0.8, 0.2])

train_dataloader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=16, shuffle=False)

trainer = dl.Trainer(max_epochs=10)

trainer.fit(classifier, train_dataloader, val_dataloader)
trainer.test(classifier, val_dataloader)

## Quality-of-Life Improvements for IDE